<a href="https://colab.research.google.com/github/GeorgeTelles/Backtest_Bollinger_Bands/blob/main/Backtesting_Bandas_de_Bollinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" alt="logo clara" width="300" style="display: inline-block; vertical-align: top; margin-right: 10px;">
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" alt="logo dark" width="300" style="display: inline-block; vertical-align: top;">
</div>

---
# **Backtesting: Bandas de Bollinger**
---

## Descrição

Este projeto tem como objetivo desenvolver um algoritmo em Python para realizar backtesting de ativos financeiros utilizando as Bandas de Bollinger. O backtesting é uma técnica essencial para avaliar a eficácia de estratégias de investimento, permitindo que os investidores testem suas abordagens com dados históricos antes de aplicá-las em tempo real.


## Funcionalidades do Projeto

- **Coleta de Dados**: Importar dados históricos de preços de ativos financeiros (ações, moedas, etc.) a partir de fontes como APIs financeiras ou arquivos CSV.
- **Cálculo das Bandas de Bollinger**: Implementar o cálculo das bandas superior, inferior e da média móvel.
- **Simulação de Estratégias**: Testar diferentes estratégias de negociação baseadas nas Bandas de Bollinger, como a estratégia de rompimento ou reversão à média.
- **Avaliação de Desempenho**: Medir o desempenho das estratégias usando métricas como retorno total, drawdown, e outras métricas financeiras relevantes.
- **Visualização**: Gerar gráficos para visualizar os preços dos ativos, as Bandas de Bollinger e os sinais de compra/venda.

## Tecnologias Utilizadas

- **Linguagem**: Python
- **Bibliotecas**:
  - `pandas` para manipulação de dados
  - `numpy` para cálculos numéricos
  - `matplotlib` e `seaborn` para visualização
  - `requests` ou `yfinance` para coleta de dados financeiros
- **Ambiente de Desenvolvimento**: Jupyter Notebook ou IDE de sua escolha

## Estrutura do Projeto

1. **Data Collection**: Scripts para importar e limpar dados financeiros.
2. **Bollinger Bands Calculation**: Implementação dos cálculos das bandas superior e inferior.
3. **Backtesting Engine**: Lógica para simulação de estratégias de negociação e avaliação de desempenho.
4. **Visualization**: Ferramentas para criar gráficos e relatórios.


**Disclaimer:**<p>
*Este código foi elaborado para fins exclusivamente demostrativos. O conteúdo aqui apresentado visa fornecer informações complementares para auxiliar o investidor na tomada de suas próprias decisões de investimento. Reforçando, nenhum tópico aqui abordado constitui qualquer tipo de indicação/oferta/solicitação de compra/venda de qualquer produto.*

Documentação da biblioteca que será usada: <p>

https://vectorbt.dev/

##O que são as bandas de bollinger

- Bandas de Bollinger são uma ferramenta popular na análise técnica usada para avaliar a volatilidade e os movimentos de preços de um ativo financeiro, como ações, moedas, commodities, entre outros. Elas foram desenvolvidas por John Bollinger na década de 1980 e consistem em três linhas que são plotadas em um gráfico de preços.

As três linhas das Bandas de Bollinger são as seguintes:

- Média Móvel Simples (MMS): É uma linha que representa a média dos preços de fechamento do ativo ao longo de um determinado período de tempo. O período padrão é geralmente definido como 20 períodos, mas os traders podem ajustá-lo conforme a sua estratégia.

- Banda Superior: Essa banda é calculada somando duas vezes o desvio padrão da média móvel simples ao preço médio. A fórmula é: Banda Superior = Média Móvel Simples + (2 * Desvio Padrão).

- Banda Inferior: Similarmente à banda superior, a banda inferior é calculada subtraindo duas vezes o desvio padrão da média móvel simples do preço médio. A fórmula é: Banda Inferior = Média Móvel Simples - (2 * Desvio Padrão).

As bandas de Bollinger se expandem ou contraem em resposta à volatilidade do mercado. Quando a volatilidade é alta, as bandas se expandem, e quando a volatilidade é baixa, elas se contraem. Isso faz com que as bandas se ajustem às mudanças de volatilidade, proporcionando uma representação visual da possível amplitude dos movimentos de preços.

#1.Instalando e importando Bibliotecas

In [ ]:
!pip install yfinance
!pip install vectorbt
!pip install ta

In [ ]:
import pandas as pd
import numpy as np

import vectorbt as vbt
import yfinance as yf
import ta

import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

#2. Capturando os dados

In [ ]:
ativo = 'PETR4.SA'
dados_ohlc = vbt.YFData.download(ativo, start='2013-03-07').get()

#3. Tratando, modelando e separando os dados

In [ ]:
proporcao_treino = 0.70

# Calcular o índice de separação
total_linhas = len(dados_ohlc)
indice_separacao = int(total_linhas * proporcao_treino)

# Dividir o DataFrame
dados_teste = dados_ohlc.iloc[:indice_separacao]
dados_valid = dados_ohlc.iloc[indice_separacao:]

# Verificar os tamanhos dos conjuntos
print(f"Tamanho do conjunto de teste: {len(dados_teste)}")
print(f"Tamanho do conjunto de validação: {len(dados_valid)}")

#4. Definindo metricas a serem testadas

In [ ]:
MA_BB =[3, 5, 7, 9, 12, 20, 21, 22, 30, 34, 50, 60, 66, 90, 100, 110, 120, 200]
windows_dev = list(range(1, 5, 1))

#5. Realizando os backtestings

In [ ]:
lista_resultados = []
lista_backtest = []

for MA in MA_BB:
    for windows in windows_dev:
        dados_teste2 = dados_teste.copy()
        bbol_ativo = ta.volatility.BollingerBands(dados_teste2.Close, window=MA, window_dev=windows)
        dados_teste2['BBol_sup'] = bbol_ativo.bollinger_hband()
        dados_teste2['BBol_inf'] = bbol_ativo.bollinger_lband()
        dados_teste2['BBol_media'] = bbol_ativo.bollinger_mavg()
        dados_teste2 = dados_teste2.dropna()

        entradas = (dados_teste2['Close'] < dados_teste2['BBol_inf']).shift(+1).fillna(False)
        saidas = (dados_teste2['Close'] > dados_teste2['BBol_sup']).shift(+1).fillna(False)

        backtest = vbt.Portfolio.from_signals(dados_teste2['Close'],
                                                    entradas,
                                                    saidas,
                                                    direction='longonly',
                                                    size_type='Amount', size=1)
        retorno_holding = (dados_teste2['Close'].iloc[-1] - dados_teste2['Close'].iloc[0]) / dados_teste2['Close'].iloc[0]

        lista_resultados.append([MA, windows, backtest.stats()['Total Return [%]'], backtest.trades.records_readable["Return"].sum()*100, backtest.stats()['Benchmark Return [%]'], retorno_holding*100 ])
        lista_backtest.append(backtest)


#6. Conferindo os 10 melhores resultados

In [ ]:
resultados = pd.DataFrame(lista_resultados, columns=['Média','Desvio', 'Resultado', "Resultado 2", 'Resultado Holding', 'Resultado Holding 2'])
top10 = resultados.sort_values(by='Resultado 2', ascending=False).head(10)
top10

#7. Visualização individual do teste

In [ ]:
lista_backtest[1].plot().show()

In [ ]:
lista_backtest[1].stats()

#8. Teste Apenas com os parametros top10 dos dados de teste

In [ ]:
medias_top = top10['Média'].tolist()
desvios_top = top10['Desvio'].tolist()

In [ ]:
lista_resultados_teste = []
lista_backtest_teste = []

for i in range(len(medias_top)):
        dados_valid2 = dados_valid.copy()
        bbol_ativo = ta.volatility.BollingerBands(dados_valid2.Close, window=medias_top[i], window_dev=desvios_top[i])
        dados_valid2['BBol_sup'] = bbol_ativo.bollinger_hband()
        dados_valid2['BBol_inf'] = bbol_ativo.bollinger_lband()
        dados_valid2['BBol_media'] = bbol_ativo.bollinger_mavg()
        dados_valid2 = dados_valid2.dropna()

        entradas = (dados_valid2['Close'] < dados_valid2['BBol_inf']).shift(+1).fillna(False)
        saidas = (dados_valid2['Close'] > dados_valid2['BBol_sup']).shift(+1).fillna(False)

        backtest = vbt.Portfolio.from_signals(dados_valid2['Close'],
                                                    entradas,
                                                    saidas,
                                                    direction='longonly',
                                                    size_type='Amount', size=1)
        retorno_holding = (dados_valid2['Close'].iloc[-1] - dados_valid2['Close'].iloc[0]) / dados_valid2['Close'].iloc[0]

        lista_resultados_teste.append([medias_top[i], desvios_top[i], backtest.stats()['Total Return [%]'], backtest.trades.records_readable["Return"].sum()*100, backtest.stats()['Benchmark Return [%]'], retorno_holding*100 ])
        lista_backtest_teste.append(backtest)

In [ ]:
resultados_teste = pd.DataFrame(lista_resultados_teste, columns=['Média','Desvio', 'Resultado', "Resultado 2", 'Resultado Holding', 'Resultado Holding 2'])
top10_teste = resultados_teste.sort_values(by='Resultado 2', ascending=False).head(10)
top10_teste

#9. Backtesting com dados de Validação

In [ ]:
lista_resultados_valid = []
lista_backtest_valid = []

for MA in MA_BB:
    for windows in windows_dev:
        dados_valid2 = dados_valid.copy()
        bbol_ativo = ta.volatility.BollingerBands(dados_valid2.Close, window=MA, window_dev=windows)
        dados_valid2['BBol_sup'] = bbol_ativo.bollinger_hband()
        dados_valid2['BBol_inf'] = bbol_ativo.bollinger_lband()
        dados_valid2['BBol_media'] = bbol_ativo.bollinger_mavg()
        dados_valid2 = dados_valid2.dropna()

        entradas = (dados_valid2['Close'] < dados_valid2['BBol_inf']).shift(+1).fillna(False)
        saidas = (dados_valid2['Close'] > dados_valid2['BBol_sup']).shift(+1).fillna(False)

        backtest = vbt.Portfolio.from_signals(dados_valid2['Close'],
                                                    entradas,
                                                    saidas,
                                                    direction='longonly',
                                                    size_type='Amount', size=1)
        retorno_holding = (dados_valid2['Close'].iloc[-1] - dados_valid2['Close'].iloc[0]) / dados_valid2['Close'].iloc[0]

        lista_resultados_valid.append([MA, windows, backtest.stats()['Total Return [%]'], backtest.trades.records_readable["Return"].sum()*100, backtest.stats()['Benchmark Return [%]'], retorno_holding*100 ])
        lista_backtest_valid.append(backtest)


In [ ]:
resultados_valid = pd.DataFrame(lista_resultados_valid, columns=['Média','Desvio', 'Resultado', "Resultado 2", 'Resultado Holding', 'Resultado Holding 2'])
top10_valid = resultados_valid.sort_values(by='Resultado', ascending=False).head(10)
top10_valid

In [ ]:
lista_backtest_valid[13].plot().show()

In [ ]:
lista_backtest_valid[13].stats()

## Conclusão do Projeto

**O projeto de backtesting utilizando Bandas de Bollinger forneceu varios insights**

### Possibilidades de Melhoria

Os resultados mostram que há oportunidades para otimizar a estratégia. Ajustar os parâmetros das Bandas de Bollinger, como o período da média móvel e o número de desvios padrão, pode impactar significativamente os resultados. Além disso, ajustar os critérios de entrada e saída, e incorporar filtros adicionais pode melhorar o desempenho da estratégia.

Recomenda-se experimentar diferentes configurações e parâmetros para adaptar a estratégia às características específicas do ativo ou do mercado em questão, buscando um melhor equilíbrio entre risco e retorno.

---

Se precisar de ajuda para ajustar a estratégia ou realizar análises adicionais, sinta-se à vontade para entrar em contato.
